In [1]:
import pandas as pd
from sklearn.cluster import SpectralClustering
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_circles
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering, KMeans

In [2]:
GloVe_df = pd.read_csv("GloVe.csv")
Comm_df = pd.read_csv("HN_community - Copy.csv")
GloVe_df = GloVe_df.query("hate != 'other'")
Users_df = pd.merge(GloVe_df, Comm_df, on = "user_id", how = "inner") 

In [3]:
community = [1227,775,1340,188,1351,613,1739,1459,1242,1709]
Users_df['Community Labels'] = Users_df['modularity_class'].apply(lambda x: 0 if x not in community else x)

In [4]:
GloVe = Users_df[Users_df.columns[2:-2]].to_numpy()
GloVe.shape

(4756, 300)

In [32]:
%%time
clustering = SpectralClustering(n_clusters=11,
        assign_labels="discretize",
        random_state=0).fit(GloVe)
Users_df['spectral'] = clustering.labels_
Users_df['spectral'].value_counts(sort=True)

Wall time: 12.7 s


1    4655
6      66
8       7
0       6
5       6
7       5
3       4
4       3
2       3
9       1
Name: spectral, dtype: int64

In [6]:
%%time
clustering = SpectralClustering(n_clusters=11,
        assign_labels="discretize",
        random_state=0).fit(GloVe)
Users_df['spectral'] = clustering.labels_
Users_df['spectral'].value_counts(sort=True)

Wall time: 13.1 s


1    4655
6      66
8       7
0       6
5       6
7       5
3       4
4       3
2       3
9       1
Name: spectral, dtype: int64

In [7]:
%%time
normalizeGloVe = GloVe / np.linalg.norm(GloVe)
clustering = SpectralClustering(n_clusters=11,
        assign_labels="discretize",
        random_state=0).fit(normalizeGloVe)
Users_df['normal_spectral'] = clustering.labels_
Users_df['normal_spectral'].value_counts(sort=True)

Wall time: 12.9 s


2     985
4     771
1     616
7     461
8     407
6     392
9     339
5     325
0     216
3     190
10     54
Name: normal_spectral, dtype: int64

In [8]:
%%time
normalizeGloVe = GloVe / np.linalg.norm(GloVe)
clustering = SpectralClustering(n_clusters=11,
        assign_labels="discretize",
        random_state=0).fit(normalizeGloVe)
Users_df['normal_spectral'] = clustering.labels_
Users_df['normal_spectral'].value_counts(sort=True)

Wall time: 12.9 s


2     985
4     771
1     616
7     461
8     407
6     392
9     339
5     325
0     216
3     190
10     54
Name: normal_spectral, dtype: int64

In [9]:
%%time
epsilon = 0.0025
minimumSamples = 300
db = DBSCAN(eps=epsilon, min_samples=minimumSamples, algorithm='ball_tree').fit(normalizeGloVe)
labels = db.labels_
print(set(labels))
Users_df['DBSCAN']=labels
Users_df['DBSCAN'].value_counts(sort=True)

{0, -1}
Wall time: 7.77 s


 0    4178
-1     578
Name: DBSCAN, dtype: int64

In [10]:
%%time
epsilon = 0.0025
minimumSamples = 300
db = DBSCAN(eps=epsilon, min_samples=minimumSamples, algorithm='ball_tree').fit(normalizeGloVe)
labels = db.labels_
print(set(labels))
Users_df['DBSCAN']=labels
Users_df['DBSCAN'].value_counts(sort=True)

{0, -1}
Wall time: 7.73 s


 0    4178
-1     578
Name: DBSCAN, dtype: int64

In [14]:
km = KMeans(init='k-means++', n_clusters=11)
km_clustering = km.fit(normalizeGloVe)
labels = km_clustering.labels_
Users_df['km1_groups']=labels
Users_df['km1_groups'].value_counts()

7     928
3     901
1     793
8     534
9     465
0     413
4     377
6     200
2      98
10     32
5      15
Name: km1_groups, dtype: int64

In [15]:
km = KMeans(init='k-means++', n_clusters=11)
km_clustering = km.fit(normalizeGloVe)
labels = km_clustering.labels_
Users_df['km2_groups']=labels
Users_df['km2_groups'].value_counts()

2     920
3     874
7     779
8     539
1     474
10    426
9     390
5     199
0     105
4      31
6      19
Name: km2_groups, dtype: int64

In [16]:
km = KMeans(init='k-means++', n_clusters=11)
km_clustering = km.fit(normalizeGloVe)
labels = km_clustering.labels_
Users_df['km3_groups']=labels
Users_df['km3_groups'].value_counts()

4     916
3     879
0     787
8     534
7     467
10    427
2     397
6     194
9     105
1      31
5      19
Name: km3_groups, dtype: int64

In [19]:
from scipy.stats import chi2_contingency
import seaborn as sns
orange = sns.light_palette("orange", as_cmap=True)

In [33]:
sp = pd.crosstab(Users_df['Community Labels'],Users_df['spectral'])
print(chi2_contingency(sp)[0:3])

sp.style.background_gradient(cmap=orange).to_excel("sp_c.xlsx")

sp.style.background_gradient(cmap=orange)

(142.15699051422874, 0.00038036770686136377, 90)


spectral,0,1,2,3,4,5,6,7,8,9
Community Labels,,,,,,,,,,
0,5,2270,1,1,2,6,42,2,4,1
188,0,342,0,1,0,0,0,0,0,0
613,0,181,1,0,0,0,0,0,1,0
775,0,350,0,0,0,0,1,0,0,0
1227,1,528,1,0,0,0,2,0,0,0
1242,0,77,0,0,0,0,1,0,0,0
1340,0,347,0,0,0,0,0,0,2,0
1351,0,258,0,2,1,0,14,3,0,0
1459,0,89,0,0,0,0,5,0,0,0


In [25]:
nsp = pd.crosstab(Users_df['Community Labels'],Users_df['normal_spectral'])
print(chi2_contingency(nsp)[0:3])

nsp.style.background_gradient(cmap=orange).to_excel("nsp_c.xlsx")

nsp.style.background_gradient(cmap=orange)

(4751.389114980437, 0.0, 100)


normal_spectral,0,1,2,3,4,5,6,7,8,9,10
Community Labels,,,,,,,,,,,
0,120,134,480,125,244,76,302,337,269,216,31
188,5,155,3,4,105,37,15,9,5,5,0
613,0,11,57,5,71,0,1,7,6,22,3
775,2,107,9,6,193,8,2,2,1,21,0
1227,8,21,348,5,36,1,5,19,71,12,6
1242,0,32,4,2,16,8,1,7,5,3,0
1340,3,92,53,7,47,17,35,42,33,14,6
1351,6,25,3,29,19,153,11,12,2,12,6
1459,72,1,0,1,4,15,1,0,0,0,0


In [26]:
km = pd.crosstab(Users_df['Community Labels'],Users_df['km1_groups'])
print(chi2_contingency(km)[0:3])

km.style.background_gradient(cmap=orange).to_excel("km_c.xlsx")

km.style.background_gradient(cmap=orange)

(3350.705820136367, 0.0, 100)


km1_groups,0,1,2,3,4,5,6,7,8,9,10
Community Labels,,,,,,,,,,,
0,313,331,65,458,106,10,169,205,409,249,19
188,6,63,0,4,28,0,2,229,7,3,1
613,1,67,2,48,1,2,3,25,5,29,0
775,4,134,2,6,17,0,0,169,3,16,0
1227,5,64,2,302,4,1,7,21,22,103,1
1242,2,14,0,4,10,0,1,34,6,6,1
1340,25,71,2,45,16,2,4,106,41,37,0
1351,32,11,11,2,131,0,4,68,5,4,10
1459,0,3,11,0,53,0,1,22,2,2,0


In [29]:
kmsp = pd.crosstab(Users_df['km1_groups'],Users_df['normal_spectral'])
print(chi2_contingency(kmsp)[0:3])

kmsp.style.background_gradient(cmap=orange).to_excel("km_sp.xlsx")

kmsp.style.background_gradient(cmap=orange)

(13289.807934505385, 0.0, 100)


normal_spectral,0,1,2,3,4,5,6,7,8,9,10
km1_groups,,,,,,,,,,,
0,1,17,0,43,4,15,238,82,1,11,1
1,4,71,43,1,499,6,12,64,40,53,0
2,56,1,0,15,0,5,1,0,2,1,17
3,4,0,827,1,31,0,1,18,5,12,2
4,82,47,0,15,0,202,12,3,2,10,4
5,0,0,0,0,0,0,0,0,0,0,15
6,2,4,0,24,0,1,22,25,64,56,2
7,29,449,0,11,227,93,54,19,9,37,0
8,2,3,0,23,5,2,49,219,124,106,1


In [30]:
spdb = pd.crosstab(Users_df['spectral'],Users_df['DBSCAN'])
print(chi2_contingency(spdb)[0:3])

spdb.style.background_gradient(cmap=orange).to_excel("km_sp.xlsx")

spdb.style.background_gradient(cmap=orange)

(745.9060533191605, 9.285120993300721e-155, 9)


DBSCAN,-1,0
spectral,,
0,6,0
1,477,4178
2,3,0
3,4,0
4,3,0
5,6,0
6,66,0
7,5,0
8,7,0
